In [1]:
from google.colab import files
import pandas as pd

# ipload the CSV file
uploaded = files.upload()

# load the dataset
pokemon_data = pd.read_csv(next(iter(uploaded)))

#use pokemon_similarity.csv

StopIteration: ignored

This program uses k-Nearest Neighbors (KNN) to find Pokémon with similar base stat distributions based on user input. The KNN model is trained on the base stats of known Pokémon, and users can input either a Pokémon's name or custom stats to find the most similar Pokémon.


In [ ]:
#Nearest stat pokemon
from google.colab import files
import pandas as pd
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler
import ipywidgets as widgets
from IPython.display import display, Image
import seaborn as sns
import matplotlib.pyplot as plt

stats_columns = ['Hp', 'Attack', 'Defense', 'Sp_Attack', 'Sp_Defense', 'Speed']
X = pokemon_data[stats_columns].values

# Normalize data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# fit a KNN model
knn_model = NearestNeighbors(n_neighbors=5, algorithm='auto', metric = 'euclidean')
knn_model.fit(X_scaled)

# create text input widgets for user input
style = {'description_width': 'initial'}
pokemon_name_input = widgets.Text(description='Enter Pokemon Name:', style=style)
hp_input = widgets.IntText(description='HP:', value=100, style=style)
attack_input = widgets.IntText(description='Attack:', value=100, style=style)
defense_input = widgets.IntText(description='Defense:', value=100, style=style)
sp_attack_input = widgets.IntText(description='Sp. Attack:', value=100, style=style)
sp_defense_input = widgets.IntText(description='Sp. Defense:', value=100, style=style)
speed_input = widgets.IntText(description='Speed:', value=100, style=style)

inputs = [hp_input, attack_input, defense_input, sp_attack_input, sp_defense_input, speed_input]

# create Submit buttons
submit_pokemon_name_button = widgets.Button(description='Submit Name', style=style)
submit_stats_button = widgets.Button(description='Submit Stats', style=style)
output = widgets.Output()


A smaller n_neighbors value would lead to a model with low bias but high variance. The predictions would be influenced by the noise in the data, making the model less robust.
A larger n_neighbors value would result in a smoother decision boundary but might introduce bias by oversimplifying the underlying patterns.

By chosing odd number for n_neighbors avoids situations where there might be a tie in voting.

Pokémon base stats often exhibit diverse patterns, and a moderate n_neighbors value allows for capturing nuanced similarities without being too sensitive to outliers or noise.
Empirical Validation:

This value provides a good balance between capturing local patterns and generalizing well to unseen data.

A smaller n_neighbors value reduces the computational cost of finding neighbors during prediction, which is advantageous, especially in interactive applications.

In [ ]:
# Find closest Pokémon based on user input
def find_closest_pokemon(user_stats):
    input_stats = [input_widget.value for input_widget in user_stats]
    input_scaled = scaler.transform([input_stats])
    _, indices = knn_model.kneighbors(input_scaled)
    return list(pokemon_data.iloc[indices[0]]['Name'])


 This function is crucial for finding the closest Pokémon based on user input, whether it's a Pokémon's name or custom stats.


It extracts the user input stats from the input widgets, representing either the entered Pokémon's base stats or custom stats.
The user inputs are collected into input_stats.
Scaling Input:

The collected stats are scaled using the previously fitted scaler to ensure consistency with the training data.
The scaled input is stored in input_scaled.
Finding Neighbors:

The KNN model is then used to find the nearest neighbors based on the scaled input.

The indices of the nearest neighbors are stored in indices.
Returning Results:

The function returns a list of Pokémon names corresponding to the closest neighbors based on the provided input.

In [ ]:

#display information for multiple Pokémon
def display_multiple_pokemon_info(names):
    for name in names:
        pokemon_info = pokemon_data[pokemon_data['Name'] == name].iloc[0]

        print(f"Pokemon: {pokemon_info['Name']}")
        display(Image(url=pokemon_info['Sprite URL'], format='png', width=100))

        print("\nPokemon Information:")
        print(f"Type 1: {pokemon_info['Type_1']}")
        print(f"Type 2: {pokemon_info['Type_2']}")

        stats = pokemon_info[['Hp', 'Attack', 'Defense', 'Sp_Attack', 'Sp_Defense', 'Speed']]
        stats.index = ['Hp', 'Attack', 'Defense', 'Sp. Attack', 'Sp. Defense', 'Speed']
        plt.figure(figsize=(8, 5))
        sns.barplot(x=stats.index, y=stats.values)
        plt.title("Pokemon Stats")
        plt.xlabel("Stat")
        plt.ylabel("Value")
        plt.xticks(rotation=45)
        plt.show()
        print("\nStats:")
        print(stats)

# function to handle submitting Pokémon name
def on_submit_pokemon_name(b):
    with output:
        output.clear_output()
        entered_name = pokemon_name_input.value.strip()
        entered_name = entered_name.capitalize()
        pokemon_info = pokemon_data[pokemon_data['Name'] == entered_name]
        if not pokemon_info.empty:

            # Extract stats for the entered Pokémon name
            stats = pokemon_info[stats_columns].values.flatten()
            input_scaled = scaler.transform([stats])
            _, indices = knn_model.kneighbors(input_scaled)
            closest_pokemon = list(pokemon_data.iloc[indices[0]]['Name'])
            display_multiple_pokemon_info(closest_pokemon)
        else:
            print("Pokemon not found. Please enter a valid Pokemon name.")

# function to handle submitting custom stats
def on_submit_stats(b):
    with output:
        output.clear_output()
        closest_pokemon = find_closest_pokemon(inputs)
        display_multiple_pokemon_info(closest_pokemon)

submit_pokemon_name_button.on_click(on_submit_pokemon_name)
submit_stats_button.on_click(on_submit_stats)

# display widgets and output
print('Choose One:')
display(pokemon_name_input)
display(submit_pokemon_name_button)
print("\nOr")
display(hp_input, attack_input, defense_input, sp_attack_input, sp_defense_input, speed_input)
display(submit_stats_button)
display(output)


Choose One:


Text(value='', description='Enter Pokemon Name:', style=DescriptionStyle(description_width='initial'))

Button(description='Submit Name', style=ButtonStyle())


Or


IntText(value=100, description='HP:', style=DescriptionStyle(description_width='initial'))

...

IntText(value=100, description='Sp. Attack:', style=DescriptionStyle(description_width='initial'))

IntText(value=100, description='Sp. Defense:', style=DescriptionStyle(description_width='initial'))

IntText(value=100, description='Speed:', style=DescriptionStyle(description_width='initial'))

Button(description='Submit Stats', style=ButtonStyle())

Output()

#Pokemon to try
# Pikachu
# Golem
# Togepi
# Charizard
# Squirtle

Explanation: This function displays detailed information for multiple Pokémon, including their name, image, types, and base stats.

Looping Through Pokémon:

It iterates through the list of Pokémon names returned by the find_closest_pokemon function.
Displaying Pokémon Information:

For each Pokémon, it prints the Pokémon's name and displays its sprite image.
Additionally, it prints the Pokémon's types  and displays a bar plot of its base stats.
on_submit_pokemon_name(b) and on_submit_stats(b)
These functions handle the logic when a user submits either a Pokémon name or custom stats.

on_submit_pokemon_name(b)

This function is triggered when the user submits a Pokémon name.
It extracts the entered name, retrieves the corresponding Pokémon's base stats, finds the closest Pokémon using the KNN model, and displays detailed information for the closest Pokémon.
on_submit_stats(b)

This function is triggered when the user submits custom stats.
It calls the find_closest_pokemon function to find the closest Pokémon based on the entered custom stats and displays detailed information for the closest Pokémon.
Widget Interaction

Buttons and Event Handling:

The submit_pokemon_name_button and submit_stats_button are buttons that, when clicked, trigger the respective submission functions.
Output Display:

The output widget is used to display the results and is cleared before updating with new information.
Display Widgets
User Interaction:

The user is presented with input widgets (pokemon_name_input, hp_input, etc.) for entering Pokémon names or custom stats.
Button Click Event Handling:

The on_click events for the buttons (submit_pokemon_name_button and submit_stats_button) are linked to the respective submission functions.
Overall Display:

The user is guided to choose between entering a Pokémon name or custom stats, enhancing the interactive and user-friendly nature of the application.
